<a href="https://colab.research.google.com/github/paritachavda/Face-Recognition-using-Sparse-Classifier/blob/main/CMU_PIE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sparse representation based classifier on CMU-PIE dataset**
*   Dataset Info: It has 65 person’s face data for 43 variable lightning scenarios along with 4 facial expressions(Total Images: 2795)
*   Training:22 images per person(1430 images)
*  Testing: 21 images per person(1365 images)
*  Original dimension:4096
*  Reduced dimension: 60
*   Accuracy: 98.90% 








In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from PIL import Image
import os, sys

In [ ]:
def read_images(path, sz=None):     
    c=0
    # count=0
    X,y = [], []
    for dirname , dirnames , filenames in os.walk(path):
        for subdirname in dirnames:
            subject_path = os.path.join(dirname , subdirname) 
            
            for filename in os.listdir(subject_path):
                    if filename.endswith('.jpg'):
                        # count=count+1
                        im = Image.open(os.path.join(subject_path , filename))
                        # print(count) 
                        if (sz is not None):
                            im = im.resize(sz, Image.NEAREST)     
                            X.append(np.asarray(im, dtype=np.uint8)) 
                            y.append(c) 
                
            
            c = c+1          
    
    return [X,y]

In [ ]:
path_images = '/content/drive/My Drive/thesis/dataset/CMU/piedb_lights' 
X1,y1 = read_images(path_images, sz=(64,64))
print("Number of loaded items ",len(X1))

Number of loaded items  1430


In [ ]:
path_images = '/content/drive/My Drive/thesis/dataset/CMU/piedb_nolights' 
X2,y2 = read_images(path_images, sz=(64,64))
print("Number of loaded items ",len(X2))

Number of loaded items  1365


In [ ]:
X = X1 + X2
y = y1 + y2

In [ ]:
import collections
my_dict = collections.defaultdict(int)
for i in y:
    my_dict[i] += 1
temp = min(my_dict.items(), key=lambda x: x[1])
print(temp)
min_num_img = temp[1]
# my_dict

(0, 43)


In [ ]:
np.random.seed(2020)

Xtrain, Xtest = [], []
ytrain, ytest = [], []

for i in range(len(X)):
    if i%43==0:
        train = np.random.choice(43,22,replace=False)
#         print(train)  
    if ((i%43 in train)):
        Xtrain.append(X[i])
        ytrain.append(y[i])        
    else:
        Xtest.append(X[i])
        ytest.append(y[i])      

len(Xtrain)

1430

In [ ]:
img_in_class = 22
number_of_classes = 65
width = 64
height = 64

training_data = np.zeros((number_of_classes*img_in_class,width*height))
training_label = np.array(ytrain)

for index in range(len(Xtrain)):
    training_data[index] = Xtrain[index].flatten()


In [ ]:
training_data_col = training_data.T

In [ ]:
training_data_col

array([[ 31., 133., 117., ...,   4.,   3.,  19.],
       [ 40., 168., 161., ...,   4.,   5.,  23.],
       [ 42., 173., 153., ...,   5.,   6.,  25.],
       ...,
       [149., 143., 170., ...,  20.,  24.,  17.],
       [148., 141., 169., ...,  22.,  27.,  20.],
       [147., 136., 165., ...,  29.,  35.,  26.]])

In [ ]:
X_toconcat_test = [np.reshape(e,(Xtest[0].shape[0]*Xtest[0].shape[1],1)) for e in Xtest]
test_data_col = np.concatenate(X_toconcat_test,axis=1)

In [ ]:
test_data_col.shape

(4096, 1365)

In [ ]:
from sklearn.decomposition import PCA

def eigenfaces(X_train,X_test,n_components=200):  
    pca = PCA(n_components=n_components, svd_solver='randomized',
      whiten=True).fit(X_train)
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    return X_train_pca, X_test_pca  

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
Xtrain_std = ss.fit_transform(training_data_col)
Xtest_std = ss.fit_transform(test_data_col)

Xtrain_PCA, Xtest_PCA = eigenfaces(Xtrain_std.T,Xtest_std.T,n_components=200)
Xtrain_col, Xtest_col = Xtrain_PCA.T, Xtest_PCA.T

In [ ]:
np.random.seed(2020)
from numpy.random import rand

alpha = 0.010
pca_components = 200
number_chosen_components = 60


p_zero = np.random.rand(pca_components,number_chosen_components)
y_tr = p_zero.transpose().dot(Xtrain_col)

In [ ]:
print(p_zero)

[[0.98627683 0.87339195 0.50974552 ... 0.9360322  0.80302764 0.69730515]
 [0.46138428 0.66243461 0.74966564 ... 0.60901857 0.45241441 0.29665953]
 [0.63061983 0.50703485 0.37025605 ... 0.39607047 0.60080567 0.46052199]
 ...
 [0.3650466  0.21767066 0.22357723 ... 0.62339902 0.45720097 0.87616655]
 [0.22183812 0.90937865 0.92639384 ... 0.86232088 0.45779093 0.2088607 ]
 [0.53720375 0.25427287 0.39369349 ... 0.10206991 0.32914715 0.73528457]]


In [ ]:
y_tr.shape

(60, 1430)

In [ ]:
y_tr

array([[-13.10987732,  -0.47508582,  -1.10519729, ...,  15.82867245,
          6.684658  ,  15.17797427],
       [-18.96266834,   0.3907273 ,  -4.87863487, ...,  15.22133665,
          3.63918946,  13.35727259],
       [-13.20073759,  -3.22586347,  -5.73936881, ...,   2.8450328 ,
         -6.35007118,  18.74033076],
       ...,
       [-11.27914853,   3.04599404,  -0.32766779, ...,  10.8936437 ,
          4.94481287,  15.98555801],
       [ -8.28539242,   0.94651035,   0.77237365, ...,   6.03973607,
         -0.67828446,   5.25654876],
       [-15.81099289,  -0.09676502,  -0.55668821, ...,   4.13900068,
         -5.38444434,  14.37724444]])

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import Lasso
from tqdm import tqdm

sp_re = np.zeros(shape=(y_tr.shape[1]-1,y_tr.shape[1]))
for i in tqdm(range(y_tr.shape[1])):  
    clf = Lasso(alpha)
    y = y_tr[:,i]
    y_tr = np.delete(y_tr,i,1)
    clf.fit(y_tr,y)
    y_tr=np.insert(y_tr,i,y,axis=1)
    sp_re[:,i] = clf.coef_

100%|██████████| 1430/1430 [03:33<00:00,  6.71it/s]


In [ ]:
mew = np.zeros((number_of_classes, pca_components))
M = np.zeros((1,pca_components))

for i in range(number_of_classes):
    xa = Xtrain_PCA[img_in_class*i:img_in_class*i+img_in_class,:]
    for j in xa:
        mew[i,:] = np.add(mew[i,:],j)
    mew[i,:] = np.divide(mew[i,:],float(len(xa)))

for i in Xtrain_PCA:
    M = np.add(M,i)
M = np.divide(M,float(Xtrain_PCA.shape[0]))

In [ ]:
normalised_wc_proj_sig = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)

for i in range(number_of_classes):
    for j in range(img_in_class):
        normalised_wc_proj_sig[i*img_in_class+j,:] = np.subtract(Xtrain_PCA[i*img_in_class+j,:],mew[i,:])


sw = np.zeros((pca_components,pca_components))

for i in range(number_of_classes):
    xa = normalised_wc_proj_sig[img_in_class*i:img_in_class*i+img_in_class,:]
    xa = xa.transpose()
    cov = np.dot(xa,xa.T)
    sw = sw + cov

normalised_class = np.ndarray(shape=(number_of_classes, pca_components), dtype=np.float64)
sb = np.zeros((pca_components,pca_components))

for i in range(number_of_classes):
    normalised_class[i,:] = np.subtract(mew[i,:],M)

sb = np.dot(normalised_class.T,normalised_class)
sb = np.multiply(sb,float(img_in_class))

In [ ]:
sw

array([[1394.11267386,   13.60939032,   -9.81961869, ...,   -3.70896771,
           9.28632288,   -6.47191482],
       [  13.60939032, 1260.26843472,   56.30589967, ...,   -5.26541228,
           6.65704028,    5.94736593],
       [  -9.81961869,   56.30589967,  868.89134384, ...,   12.02518101,
          15.05600625,  -15.62380406],
       ...,
       [  -3.70896771,   -5.26541228,   12.02518101, ..., 1316.29524207,
          39.14900655,  -21.95815406],
       [   9.28632288,    6.65704028,   15.05600625, ...,   39.14900655,
        1314.06306201,   23.57003419],
       [  -6.47191482,    5.94736593,  -15.62380406, ...,  -21.95815406,
          23.57003419, 1308.53012049]])

In [ ]:
sb

array([[ 34.88732614, -13.60939032,   9.81961869, ...,   3.70896771,
         -9.28632288,   6.47191482],
       [-13.60939032, 168.73156528, -56.30589967, ...,   5.26541228,
         -6.65704028,  -5.94736593],
       [  9.81961869, -56.30589967, 560.10865616, ..., -12.02518101,
        -15.05600625,  15.62380406],
       ...,
       [  3.70896771,   5.26541228, -12.02518101, ..., 112.70475793,
        -39.14900655,  21.95815406],
       [ -9.28632288,  -6.65704028, -15.05600625, ..., -39.14900655,
        114.93693799, -23.57003419],
       [  6.47191482,  -5.94736593,  15.62380406, ...,  21.95815406,
        -23.57003419, 120.46987951]])

In [ ]:
def delta(x,i,classs):
    
    n,m = len(x),len(classs)
    
    if (n != m):
        print('vectors of differents sizes, cannot operate delta')
        
    tmp = i*np.ones(n)-classs

    for k in range(n):
        if tmp[k]==0:
            tmp[k]=1
        else:
            tmp[k]=0 
            
    return tmp*x

def residu(y,A,x,class_x):
    
    k = np.max(class_x)+1
    r = np.zeros(k)
    
    for i in range(0,k):
        r[i] = np.linalg.norm(y - np.dot(A,delta(x,i,class_x)))
        
    return r

def cost(proj):
    multi_b = np.dot(proj.T,np.dot(sb+Eb,proj))
    numerator = np.trace(multi_b)
    multi_w = np.dot(proj.T,np.dot(Ew+sw,proj))
    denomenator = np.trace(multi_w)
    final_val = numerator/denomenator
    return final_val

In [ ]:
y_tr.shape

(60, 1430)

In [ ]:
num_tr = y_tr.shape[1]
Ew = np.zeros((pca_components,pca_components))
Withinclass_error = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in range(num_tr):
    x = Xtrain_col[:,i]
    label = training_label[i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    training_label = np.delete(training_label,i)
    Withinclass_error[i,:] = x-np.dot(Xtrain_col,delta(sp_re[:,i],label,training_label))
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
    training_label=np.insert(training_label,i,label)
    
Ew = np.dot(Withinclass_error.T,Withinclass_error)
Ew = Ew/number_of_classes*img_in_class

In [ ]:
Ew

array([[ 77.09164592,   1.79924839,  -3.82423469, ...,   3.55960996,
          8.7231235 ,   0.57746173],
       [  1.79924839, 140.8718602 ,   5.30938427, ...,  -0.28900886,
          3.40639385,  10.99329451],
       [ -3.82423469,   5.30938427,  32.15174141, ...,  -3.14134659,
         -3.58699878,  -1.64847609],
       ...,
       [  3.55960996,  -0.28900886,  -3.14134659, ..., 252.68170987,
         26.15073539, -21.61782406],
       [  8.7231235 ,   3.40639385,  -3.58699878, ...,  26.15073539,
        274.43746502,  16.27763682],
       [  0.57746173,  10.99329451,  -1.64847609, ..., -21.61782406,
         16.27763682, 251.06771301]])

In [ ]:
from tqdm import tqdm
num_data = number_of_classes*img_in_class

classLabels = np.unique(ytrain)
k={}
for label in classLabels:  
    temp = np.where(training_label!=label)
    k.update({label:temp})


In [33]:
Eb = np.zeros((pca_components,pca_components))
Betweenclass_error = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in tqdm(range(num_tr)):
    tmp = np.zeros((pca_components))
    x = Xtrain_col[:,i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    label = training_label[i]
    other_label_index=(k[label][0])
    for j in range(other_label_index.shape[0]):
        other_label = training_label[j]
        training_label = np.delete(training_label,j)
        tmp += x-(np.dot(Xtrain_col,delta(sp_re[:,j],other_label,training_label)))
        training_label=np.insert(training_label,j,other_label)
    Betweenclass_error[i,:] = tmp
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
Eb = np.dot(Betweenclass_error.T,Betweenclass_error)
Eb = Eb/(num_data*(number_of_classes-1))

100%|██████████| 1430/1430 [59:00<00:00,  2.48s/it]


In [34]:
Eb

array([[ 3.09598591e+04,  1.44133927e+01, -4.95856491e+00, ...,
         8.25919146e+00, -6.20195129e+00, -1.98955354e-01],
       [ 1.44133927e+01,  3.09678119e+04,  2.57887095e+01, ...,
        -3.78582838e+01,  4.94730410e+00, -3.75605258e+01],
       [-4.95856491e+00,  2.57887095e+01,  3.09570759e+04, ...,
        -1.69008536e+01, -1.19012182e+01, -1.39475666e+01],
       ...,
       [ 8.25919146e+00, -3.78582838e+01, -1.69008536e+01, ...,
         3.09753408e+04,  5.62481356e+00,  3.17532619e-01],
       [-6.20195129e+00,  4.94730410e+00, -1.19012182e+01, ...,
         5.62481356e+00,  3.09814575e+04, -4.14571601e+00],
       [-1.98955354e-01, -3.75605258e+01, -1.39475666e+01, ...,
         3.17532619e-01, -4.14571601e+00,  3.09720186e+04]])

In [36]:
def test(proj_mtx,training_label):
    training_label=training_label.astype(int)
    y_tr = proj_mtx.transpose().dot(Xtrain_col)
    sp_re_test = np.zeros((num_tr,len(Xtest)))
    y_te = proj_mtx.transpose().dot(Xtest_col)

    # print('Finding Sparse repre.:....')
    for i in tqdm(range(y_te.shape[1])):    
        clf = Lasso(alpha=0.01)
        y = y_te[:,i]
        clf.fit(y_tr,y)
        sp_re_test[:,i] = clf.coef_


    pred_class_test = np.zeros(y_te.shape[1])
    for i in tqdm(range(y_te.shape[1])):
        y = y_te[:,i]
        label = ytest[i]
        pred_class_test[i] = np.argmin(residu(y,y_tr,sp_re_test[:,i],training_label))

    pred_class_test = pred_class_test.astype(int)
    correct_prediction = 0
    for i in range(y_te.shape[1]):
        if(ytest[i]==pred_class_test[i]):
            correct_prediction=correct_prediction+1

    print('Correct Prediction:',correct_prediction)
    print('Accuracy:',correct_prediction/y_te.shape[1]*100)


In [37]:
J_dp = np.dot(np.linalg.inv((Ew)),((Eb)))
eigenvalues_dp, eigenvectors_dp, = np.linalg.eig(J_dp)
eig_pairs_dp = [(eigenvalues_dp[index], eigenvectors_dp[:,index]) for index in range(len(eigenvalues_dp))]
eig_pairs_dp.sort(key=lambda tup:tup[0],reverse=True)
eigvalues_sort_dp  = [eig_pairs_dp[index][0] for index in range(len(eigenvalues_dp))]
eigvectors_sort_dp = [eig_pairs_dp[index][1] for index in range(len(eigenvalues_dp))]
print('Number of eigen vectors:',len(eigvalues_sort_dp))
print("k:",number_chosen_components)
proj_mtx_dp = np.array(eigvectors_sort_dp[:number_chosen_components]).transpose()
correct_prediction_dp = test(proj_mtx_dp,training_label)

Number of eigen vectors: 200
k: 60


100%|██████████| 1365/1365 [02:33<00:00,  8.89it/s]

Correct Prediction: 1328
Accuracy: 97.28937728937728


In [38]:
beta = 1
J = np.dot(np.linalg.inv(sw+(beta*Ew)),(sb+(beta*Eb)))
eigenvalues, eigenvectors, = np.linalg.eig(J)
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

eig_pairs.sort(key=lambda tup:tup[0],reverse=True)
eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]
print('Number of eigen vectors:',len(eigvalues_sort))
print("k:",number_chosen_components)
proj_mtx = np.array(eigvectors_sort[:number_chosen_components]).transpose()

Number of eigen vectors: 200
k: 60


In [39]:
test(proj_mtx,training_label)

100%|██████████| 1365/1365 [02:32<00:00,  8.93it/s]

Correct Prediction: 1350
Accuracy: 98.9010989010989


In [40]:
# proj_mtx = np.array(eigvectors_sort[:number_chosen_components]).transpose()
p1 = np.zeros((pca_components,int(number_chosen_components/2)))
index = 0
end = int(number_chosen_components/2) 
for i in range(0,end):
    p1[:,index] = proj_mtx[:,i]+proj_mtx[:,end+i]/2
    # print(i)
    index+=1
test(p1,training_label)

100%|██████████| 1365/1365 [02:27<00:00,  9.25it/s]

Correct Prediction: 1240
Accuracy: 90.84249084249085


In [41]:
def training(p_zero,proj_mtx,Xtrain_col,training_label):
 
    p_k = proj_mtx.real
    p_k_1 = p_zero.real
    eta = 0.0001
    while cost(p_k)-cost(p_k_1) > eta:
        y_tr = p_k.transpose().dot(Xtrain_col)
        sp_re = np.zeros(shape=(y_tr.shape[1]-1,y_tr.shape[1]))
        for i in range(y_tr.shape[1]):  
            clf = Lasso(alpha)
            y = y_tr[:,i]
            y_tr = np.delete(y_tr,i,1)
            clf.fit(y_tr,y)
            y_tr=np.insert(y_tr,i,y,axis=1)
            sp_re[:,i] = clf.coef_

      
        num_tr = y_tr.shape[1]

        Ew = np.zeros((pca_components,pca_components))
        Withinclass_error = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
        for i in range(num_tr):
            x = Xtrain_col[:,i]
            label = training_label[i]
            Xtrain_col = np.delete(Xtrain_col,i,1)
            training_label = np.delete(training_label,i)
            Withinclass_error[i,:] = x-np.dot(Xtrain_col,delta(sp_re[:,i],label,training_label))
            Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
            training_label=np.insert(training_label,i,label)

        Ew = np.dot(Withinclass_error.T,Withinclass_error)
        Ew = Ew/number_of_classes*img_in_class

      # cal Eb
        Eb = np.zeros((pca_components,pca_components))
        Betweenclass_error = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
        for i in tqdm(range(num_tr)):
            tmp = np.zeros((pca_components))
            x = Xtrain_col[:,i]
            Xtrain_col = np.delete(Xtrain_col,i,1)
            label = training_label[i]
            other_label_index=(k[label][0])
            for j in range(other_label_index.shape[0]):
                other_label = training_label[j]
                training_label = np.delete(training_label,j)
                tmp += x-(np.dot(Xtrain_col,delta(sp_re[:,j],other_label,training_label)))
                training_label=np.insert(training_label,j,other_label)
            Betweenclass_error[i,:] = tmp
            Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
        Eb = np.dot(Betweenclass_error.T,Betweenclass_error)
        Eb = Eb/(num_data*(number_of_classes-1))
        
        J = np.dot(np.linalg.inv(sw+(beta*Ew)),(sb+(beta*Eb)))
        eigenvalues, eigenvectors, = np.linalg.eig(J)
        eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

        eig_pairs.sort(key=lambda tup:tup[0],reverse=True)
        eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
        eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]
        proj_mtx = np.array(eigvectors_sort[:number_chosen_components]).transpose()    
        p_k_1 = p_k
        p_k = proj_mtx.real
    return p_k   


In [ ]:
sp_re = np.zeros(shape=(Xtrain_col.shape[1]-1,Xtrain_col.shape[1]))
for i in range(Xtrain_col.shape[1]):  
    clf = Lasso(alpha)
    y = Xtrain_col[:,i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    clf.fit(Xtrain_col,y)
    Xtrain_col=np.insert(Xtrain_col,i,y,axis=1)
    sp_re[:,i] = clf.coef_

num_tr = Xtrain_col.shape[1]

Ew_new = np.zeros((pca_components,pca_components))
Withinclass_error_new = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in range(num_tr):
    x = Xtrain_col[:,i]
    label = training_label[i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    training_label = np.delete(training_label,i)
    Withinclass_error_new[i,:] = x-np.dot(Xtrain_col,delta(sp_re[:,i],label,training_label))
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
    training_label=np.insert(training_label,i,label)

Ew_new = np.dot(Withinclass_error_new.T,Withinclass_error_new)
Ew_new = Ew_new/number_of_classes*img_in_class

 
Eb_new = np.zeros((pca_components,pca_components))
Betweenclass_error_new = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in tqdm(range(num_tr)):
    tmp = np.zeros((pca_components))
    x = Xtrain_col[:,i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    label = training_label[i]
    other_label_index=(k[label][0])
    for j in range(other_label_index.shape[0]):
        other_label = training_label[j]
        training_label = np.delete(training_label,j)
        tmp += x-(np.dot(Xtrain_col,delta(sp_re[:,j],other_label,training_label)))
        training_label=np.insert(training_label,j,other_label)
    Betweenclass_error_new[i,:] = tmp
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
Eb_new = np.dot(Betweenclass_error_new.T,Betweenclass_error_new)
Eb_new = Eb_new/(num_data*(number_of_classes-1))


 95%|█████████▌| 1364/1430 [55:55<02:35,  2.36s/it]

In [ ]:
beta = 1
J = np.dot(np.linalg.inv(sw+(beta*Ew_new)),(sb+(beta*Eb_new)))
eigenvalues, eigenvectors, = np.linalg.eig(J)
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

eig_pairs.sort(key=lambda tup:tup[0],reverse=True)
eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]
p_zero_1 = np.array(eigvectors_sort[:number_chosen_components]).transpose() 

In [ ]:
test(p_zero_1,training_label)

In [48]:
# proj_mtx = np.array(eigvectors_sort[:number_chosen_components]).transpose()
p1 = np.zeros((pca_components,int(number_chosen_components/2)))
index = 0
end = int(number_chosen_components/2) 
for i in range(0,end):
    p1[:,index] = p_zero_1[:,i]+p_zero_1[:,end+i]/2
    # print(i)
    index+=1
test(p1,training_label)

100%|██████████| 1365/1365 [02:25<00:00,  9.37it/s]

Correct Prediction: 1247
Accuracy: 91.35531135531136
